In [ ]:
# ==========================================
# FINAL DEMO: TEST THE MODEL MANUALLY
# ==========================================
import pandas as pd
import numpy as np
import pickle
import os

def interactive_demo():
  model_path = 'taxi_duration_model.pkl'
  data_path = 'taxi_test_data.csv'

  # 1. Check if files exist
  if not os.path.exists(model_path) or not os.path.exists(data_path):
    print("❌ Error: Model or Test Data file not found.")
    print("Please ensure 'taxi_duration_model.pkl' and 'taxi_test_data.csv' are in the folder.")
    return

  # 2. Load the artifacts
  with open(model_path, 'rb') as f:
    model = pickle.load(f)
  test_df = pd.read_csv(data_path, index_col=0)

  print("✅ System Ready.")
  print(f"Total test samples available: {len(test_df)}")
  print("-" * 30)

  # 3. User Interaction
  try:
    user_input = input(f"Enter a row index to test (0 to {len(test_df)-1}): ")
    idx = int(user_input)

    # Extract the row
    row = test_df.iloc[idx]
    actual_val = row['actual_trip_duration_sec']

    # Prepare features for prediction (drop the target column we added for the CSV)
    # Note: Ensure features are in the exact same order as when the model was trained
    features = row.drop('actual_trip_duration_sec').values.reshape(1, -1)

    # 4. Predict
    pred_log = model.predict(features)
    pred_val = np.expm1(pred_log)[0] # Back to seconds

    # 5. Output Results
    print("\n" + "═"*45)
    print(f" 🚕 TRIP PREDICTION RESULTS (Index: {idx})")
    print("═"*45)
    print(f" 📍 Distance:         {row['haversine_distance_km']:.2f} km")
    print(f" ⏰ Pickup Hour:      {int(row['pickup_hour'])}:00")
    print(f" 📅 Day of Week:      {int(row['day_of_week'])} (0=Mon, 6=Sun)")
    print(f" 🏁 Holiday/Weekend: {'Yes' if row['is_weekend'] + row['is_holiday'] > 0 else 'No'}")
    print("-" * 45)
    print(f" ⏱️ ACTUAL TIME:     {actual_val/60:.2f} minutes")
    print(f" 🤖 PREDICTED TIME:  {pred_val/60:.2f} minutes")
    print("-" * 45)

    error = abs(actual_val - pred_val) / 60
    accuracy = (1 - (abs(actual_val - pred_val) / actual_val)) * 100
    print(f" 📝 Deviation:       {error:.2f} minutes")
    print(f" 🎯 Accuracy:        {max(0, accuracy):.1f}%")
    print("═"*45)

  except ValueError:
    print("❌ Invalid input! Please enter a whole number.")
  except IndexError:
    print(f"❌ Index out of bounds! Choose between 0 and {len(test_df)-1}.")

# Execute the demo
interactive_demo()

✅ System Ready.
Total test samples available: 285737
------------------------------
